In [4]:
from pyplasm import *
import csv
import random
import time
%run "../2016-11-25/workshop_07.ipynb"
%run "../2016-10-28/Workshop_03Variante.ipynb"
%run "../tools.ipynb"

In [ ]:
# Texture
pavimento = "./texture/floor.jpg"
groundFloor = "./texture/ground.jpg"

In [5]:
"""This function uses the constants above and creates an HPC object of an house"""

def Apartment(flag):
    # HPC
    externalWalls = readFile(pathExternalWalls)
    internalWalls = readFile(pathInternalWalls)
    grind = readFile(pathGrindWalls)
    aperture = readFile(pathAperture)
    main = readFile(pathMainEntrance)
    stairFloor = readFile(pathStairs2)
    ground = readFile(pathFloor)
    balconyFloor = readFile(pathBalconyFloor)
    
    # OFFSET
    externalWalls = OFFSET([externalWallsThickness,externalWallsThickness])(externalWalls)
    internalWalls = OFFSET([internalWallsThickness,internalWallsThickness])(internalWalls)
    grind = OFFSET([grindWallsThickness,grindWallsThickness])(grind)
    aperture = OFFSET([externalWallsThickness+1, externalWallsThickness+1])(aperture)
    
    # PROD
    externalWalls = PROD([externalWalls, QUOTE([houseHeight])])
    internalWalls = PROD([internalWalls, QUOTE([houseHeight])])
    grind = PROD([grind, QUOTE([grindHeight])])
    aperture = PROD([aperture, QUOTE([doorsHeight])])
    
    # BOX
    windowBoxs = readWindowBox(pathWindows, wx, externalWallsThickness, wz, wb)
    doorBoxs = readDoorBox(pathDoors, externalWallsThickness, doorsHeight)
    mainBox = readMainDoorBox(pathMainEntrance, externalWallsThickness, doorsHeight)
    
    # OBJECT
    windows = readWindows(pathWindows, wx, wy, wz, wb)
    doors = readDoor(pathDoors, 1, doorsHeight)
    stairFloor = SOLIDIFY(stairFloor)
    ground = TEXTURE([groundFloor, TRUE, TRUE,10,10,0,1,1,0,0])(SOLIDIFY(ground))
    main = readMainDoor(pathMainEntrance, 1, doorsHeight)
    stairs = readStairs(pathStairs, 4)
    stairs = scale(115.2, 58, 48, stairs) #108
    stairs01 = T([1,2])([675-108, 819+5])(stairs)
    stairs02 = T([1,2])([567, 162+5])(stairs)
    
    # DIFF
    grind = DIFF([grind, externalWalls])
    internalWalls = DIFF([internalWalls, externalWalls])  
    internalWalls = DIFF([internalWalls, doorBoxs])
    externalWalls = DIFF([externalWalls, windowBoxs])
    externalWalls = DIFF([externalWalls, doorBoxs])
    externalWalls = DIFF([externalWalls, aperture])
    
    if(flag):
        externalWalls = DIFF([externalWalls, mainBox])
    else:
        ground = TEXTURE([pavimento, TRUE, TRUE,10,10,0,10,10,0,0])(DIFF([ground, stairFloor]))
    
    # COLOR AND TEXTURE
    grind = COLOR(GREY)(grind)
    externalWalls = COLOR(GREY)(externalWalls)    
    internalWalls = COLOR(GREENLIGHT)(internalWalls)
    ground = TEXTURE([groundFloor, TRUE, TRUE,10,10,0,10,10,0,0])(ground)#COLOR(DARKBROWN)(ground)
    balconyFloor = COLOR(GREYCYANO)(SOLIDIFY(balconyFloor))
    
    # FINAL STRUCTURE
    struct = []
    if(flag):
        struct = STRUCT([ground, externalWalls, internalWalls, doors, windows, main, stairs01, stairs02])
    else:
        struct = STRUCT([ground , balconyFloor, externalWalls, internalWalls, grind, doors, windows, stairs01, stairs02])    
    
    # SCALA DI UN DECIMO
    struct = S([1,2,3])([0.1, 0.1, 0.1])(struct)
    return struct
